In [1]:
import torch
import torch.nn.functional as F

by applying a linear transformation to the original input vector. In other words, we add three k×k weight matrices 𝐖q, 𝐖k,𝐖v and compute three linear transformations of each xi, for the three different parts of the self attention:

𝐪i=𝐖q𝐱i𝐤i=𝐖k𝐱i𝐯i=𝐖v𝐱i
w′ijwij𝐲i=𝐪iT𝐤j=softmax(w′ij)=∑jwij𝐯j.

In [ ]:
# Implementing own attention 

class Transformer():
    
    
    def __init__():
        pass

    def self_attention(self, x):
        """Function to apply self attention to the vector x.
        Arguments
        -------------------------
        Shape of x: batch_size, t, k where t is the no. of vectors 
                    and k is the size of each vector

        Returns:
        --------------------------
        
        
        Notes:
        --------------------------
        Assume we have a tensor of size (batch_size, t vectors, k (dimension of each vector))
        k is fixed as the size of the embedding layer and the encoding layer is fixed
        so for each vector we have the same size.
        """
        # torch.bmm is batched matrix multiplication
        # this is basically xx' for calculating weights wij.
        raw_weights = torch.bmm(x, x.transpose(1,2))
        
        # Turning these weights into probabilites by applying row-wise softmax
        weights = F.softmax(raw_weights, dim=2)
        
        # y = wx to get the output vector of size (b, t, k)
        y = torch.bmm(weights,x)
        
        return y
    